<div class="usecase-title">Self Designed Walking Tour</div>

<div class="usecase-authors"><b>Authored by: </b> Amy Tran</div>

<div class="usecase-duration"><b>Duration:</b> 60 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

As international and domestic tourists are returning to Melbourne after Covid, a tool that helps visitors create their own walking experience is beneficial. Based on the visitor’s preference, the tool will suggest appropriate walking route that incorporates landmarks or places of interest. 

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- Load relevant datasets required for analysis
- Map city landmark

<div class="usecase-section-header">Python Libraries</div>

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance

#pip install plotly==5.8.0
#pip install geopy

<div class="usecase-section-header">Data Extraction</div>

In [4]:
def get_data(base, url, size = 0):
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [11]:
#Extract laneways-with-greening-potential data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'self-guided-walks'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
walks = data.copy()
#Rename Longitude and Latitude columns
walks = walks.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Completed:', walks.shape[0], 'rows,', walks.shape[1], 'columns extracted' )

Completed: 8 rows, 9 columns extracted


In [12]:
walks

,distance,name,ref,time,lon,lat,geo_shape.type,geo_shape.geometry.coordinates,geo_shape.geometry.type
0,3.5,Melbourne Music Walk,2,1.5,144.968357,-37.813502,Feature,"[[[144.968808624716, -37.821216567104], [144.9...",MultiLineString
1,3.35,The Cosmopolitan,5,2 Hours,144.968310,-37.813787,Feature,"[[144.9676848541881, -37.81761724626771], [144...",LineString
2,4.19,A Walk in the Park,1,2 Hours,144.974801,-37.814702,Feature,"[[[144.967542565756, -37.817302659845], [144.9...",MultiLineString
3,5.48,Elegant Enclave,7,2,144.979662,-37.814751,Feature,"[[[144.969044079955, -37.817186640655], [144.9...",MultiLineString
4,4.75,Sports & Entertainment,6,2 Hours,144.976500,-37.823276,Feature,"[[144.96769834830815, -37.81762413706883], [14...",LineString
5,5.24,On the Waterfront,3,2 Hours,144.959143,-37.821733,Feature,"[[[144.967996616066, -37.818218223256], [144.9...",MultiLineString
6,2.26,Arcades & Lanes,4,1.5 Hours,144.964261,-37.815192,Feature,"[[[144.967363582879, -37.817821881871], [144.9...",MultiLineString
7,5.61,Secret Gardens,2,2.5 Hours,144.974916,-37.825217,Feature,"[[144.96768788280298, -37.81762805497954], [14...",LineString
